# Reliability Analysis

In [13]:
from dataclasses import dataclass
import numpy as np
from scipy.special import expit

1) What is p?

2) The Data looks like this.

In [10]:
@dataclass
class Observation:
    broke: bool
    days: int # include the day it broke as well
        
    def __str__(self):
        return f'{self.days} {self.broke}'
        
def gen_data(p, max_n): # p is survival prob
    for i in range(1, max_n+1):
        
        if np.random.random() > p:
            return Observation(broke=True, days=i)
    return Observation(broke=False, days=max_n)

np.random.seed(555)
p = 1-0.0123 # pretend you didn't see this
data = [gen_data(p, 100) for i in range(200)]

3) Try Plot PDF P(break on day d)

4) How to we fix the fact the the graph is very spotty? What is the better thing to plot?

5) If you just average the day it breaks what happen?

6) Why is regression on time to fail a bad idea?

7) How to find p? Find it. Find p along with MTBF.

8) Wear and Tear and other features. How do we incoporate wear and tear in our model?

9) Find w

In [47]:
from typing import List

@dataclass
class DailyData:
    day: int
    broke: bool

@dataclass
class Observation:
    broke: bool
    days: int # include the day it broke as well
        
    def __str__(self):
        return f'{self.days} {self.broke}'

    def to_daily_data(self) -> List[DailyData]:
        ret = []
        for i in range(1, self.days): # every right before last day
            ret.append(DailyData(i, False))
        ret.append(DailyData(self.days, self.broke))
        return ret
            
    
def p(t):
    return expit(-t*0.015 +4) # again pretend you didn't see this

def gen_data(p, max_n): # p is survival prob
    for i in range(1, max_n+1):
        if np.random.random() > p(i):
            return Observation(broke=True, days=i)
    return Observation(broke=False, days=max_n)

np.random.seed(555)
data = [gen_data(p, 100) for i in range(200)]

In [46]:
len(data)

200

In [57]:
daily_data = sum((d.to_daily_data() for d in data), [] )
len(daily_data)

daily_data

[DailyData(day=1, broke=False),
 DailyData(day=2, broke=False),
 DailyData(day=3, broke=False),
 DailyData(day=4, broke=False),
 DailyData(day=5, broke=False),
 DailyData(day=6, broke=False),
 DailyData(day=7, broke=False),
 DailyData(day=8, broke=False),
 DailyData(day=9, broke=False),
 DailyData(day=10, broke=False),
 DailyData(day=11, broke=False),
 DailyData(day=12, broke=False),
 DailyData(day=13, broke=False),
 DailyData(day=14, broke=False),
 DailyData(day=15, broke=False),
 DailyData(day=16, broke=False),
 DailyData(day=17, broke=False),
 DailyData(day=18, broke=False),
 DailyData(day=19, broke=False),
 DailyData(day=20, broke=False),
 DailyData(day=21, broke=False),
 DailyData(day=22, broke=False),
 DailyData(day=23, broke=False),
 DailyData(day=24, broke=False),
 DailyData(day=25, broke=False),
 DailyData(day=26, broke=False),
 DailyData(day=27, broke=False),
 DailyData(day=28, broke=False),
 DailyData(day=29, broke=False),
 DailyData(day=30, broke=False),
 DailyData(day=31, 

In [55]:
from sklearn.linear_model import LogisticRegression

In [64]:
lr = LogisticRegression()

sk_data = np.array([[float(d.day)] for d in daily_data])
sk_class = np.array([not d.broke for d in daily_data])
lr.fit(sk_data, sk_class)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
lr.coef_, lr.intercept_

(array([[-0.01087961]]), array([3.75975959]))

10) What is MTBF now?

11) What is the p for day = 30?